In [1]:
from dotenv import load_dotenv
from pydantic import BaseModel, GetJsonSchemaHandler
from pydantic.json_schema import JsonSchemaValue
from typing import Any, Optional
import pandas as pd
import matplotlib.pyplot as plt
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage


load_dotenv()

from fast_flights import FlightData, Passengers, Result, get_flights
from pprint import pprint


In [2]:
from fast_flights import FlightData, Passengers, Result, get_flights
from pprint import pprint

result: Result = get_flights(
    flight_data=[
        FlightData(date="2025-05-01", from_airport="PHL", to_airport="PHX")
    ],
    trip="one-way",
    seat="economy",
    passengers=Passengers(adults=2, children=1, infants_in_seat=0, infants_on_lap=0),
    fetch_mode="fallback",
)

def format_flight(flight: FlightData):
    return {
        "name": flight.name,
        "departure": flight.departure,
        "arrival": flight.arrival,
        "duration": flight.duration,
        "stops": flight.stops,
        "price": flight.price
    }
pprint([format_flight(flight) for flight in result.flights if flight.is_best])

# The price is currently... low/typical/high
print("The price is currently", result.current_price)

[{'arrival': '1:28 PM on Thu, May 1',
  'departure': '5:15 AM on Thu, May 1',
  'duration': '11 hr 13 min',
  'name': 'Spirit',
  'price': '$326',
  'stops': 1},
 {'arrival': '2:06 PM on Thu, May 1',
  'departure': '8:41 AM on Thu, May 1',
  'duration': '8 hr 25 min',
  'name': 'Frontier',
  'price': '$357',
  'stops': 1},
 {'arrival': '11:42 AM on Thu, May 1',
  'departure': '9:27 AM on Thu, May 1',
  'duration': '5 hr 15 min',
  'name': 'American',
  'price': '$866',
  'stops': 0}]
The price is currently typical


In [3]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console

In [12]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini-2024-07-18")


In [13]:
async def lookup_hotel(location: str) -> str:
    return f"Here are some hotels in {location}: hotel1, hotel2, hotel3."

hotel_agent = AssistantAgent(
        "Hotel_Agent",
        model_client,
        tools=[lookup_hotel],
        description="Helps with hotel booking.",
        system_message="""
        You are a hotel search agent.
        Your only tool is lookup_hotel - use it to find hotels near a location.
        You only make one search at a time.
        """
    )

In [22]:
from fast_flights import FlightData, Passengers, Result, get_flights

def format_flight(flight: FlightData):
    return {
        "name": flight.name,
        "departure": flight.departure,
        "arrival": flight.arrival,
        "duration": flight.duration,
        "stops": flight.stops,
        "price": flight.price
    }


async def lookup_flight(departure_date: str, return_date: str, source_airport_code: str, destination_airport_code: str) -> str:
    leave_result: Result = get_flights(
    flight_data=[
            FlightData(date=departure_date, from_airport=source_airport_code, to_airport=destination_airport_code),
        ],
        trip="one-way",
        seat="economy",
        passengers=Passengers(adults=1, children=0, infants_in_seat=0, infants_on_lap=0),
        fetch_mode="fallback",
    )

    best_leave = format_flight([flight for flight in leave_result.flights if flight.is_best][0])

    return_result: Result = get_flights(
    flight_data=[
            FlightData(date=return_date, from_airport=destination_airport_code, to_airport=source_airport_code),
        ],
        trip="one-way",
        seat="economy",
        passengers=Passengers(adults=1, children=0, infants_in_seat=0, infants_on_lap=0),
        fetch_mode="fallback",
    )

    best_return = format_flight([flight for flight in return_result.flights if flight.is_best][0])

    res = f"Here is the best flight from {source_airport_code} to {destination_airport_code}:\n"
    for key in best_leave:
        res += f"{key}: {best_leave[key]}\n"
    res += f"Here is the best return flight from {destination_airport_code} to {source_airport_code}:\n"
    for key in best_return:
        res += f"{key}: {best_return[key]}\n"
    return res

print(await lookup_flight("2025-05-01", "2025-05-10", "PHL", "PHX"))

flight_agent = AssistantAgent(
    "Flight_Agent",
    model_client,
    tools=[lookup_flight],
    description="Helps with flight booking.",
    system_message="""
    You are a flight search agent.
    Your only tool is lookup_flight. This tool takes in a departure date in YYYY-MM-DD format, a return date in YYYY-MM-DD format, the source airport code, and the destination code, and returns .
    You may only make one search at a time.
    """
)



Here is the best flight from PHL to PHX:
name: Spirit
departure: 5:15 AM on Thu, May 1
arrival: 1:28 PM on Thu, May 1
duration: 11 hr 13 min
stops: 1
price: $109
Here is the best return flight from PHX to PHL:
name: Frontier
departure: 9:35 AM on Sat, May 10
arrival: 9:15 PM on Sat, May 10
duration: 8 hr 40 min
stops: 1
price: $90



In [17]:
async def lookup_excursion(location: str) -> str:
        return f"Here are some activities to do near {location}: thing1, thing2, thing3."

excursion_agent = AssistantAgent(
        "Excursion_Agent",
        model_client,
        tools=[lookup_excursion],
        description="Helps with finding excursions.",
        system_message="""
        You are a flight search agent.
        Your only tool is lookup_excursion - use it to find excursions near a location.
        You may only make one search at a time.
        """
)

In [23]:
# async def book_trip() -> str:
#     return "Your trip is booked!"


travel_planning_agent = AssistantAgent(
    "TravelPlanningAgent",
    model_client=model_client,
    description="Helps with travel planning.",
    system_message="""
    You are a travel planning agent,
    Your job is to break down complex tasks into smaller, manageable substasks.
    Your team members are:
    Flight agent: searches for flights
    Hotel agent: searches for hotels
    Excursion agent: searches for activities

    You only plan and delegate tasks. You do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all the tasks are complete, summarize the findings and end with "TERMINATE"
    """
)



termination = TextMentionTermination("TERMINATE")
team = SelectorGroupChat(
    [travel_planning_agent, hotel_agent, flight_agent, excursion_agent],
    model_client=model_client,
    termination_condition=termination,
)
await Console(team.run_stream(task="I live near New York City. Book me a 3 day trip in Los Angeles. I am leaving on 5/13/2025."))



---------- user ----------
I live near New York City. Book me a 3 day trip in Los Angeles. I am leaving on 5/13/2025.
---------- TravelPlanningAgent ----------
To plan your 3-day trip to Los Angeles starting on May 13, 2025, I will break down the tasks into smaller, manageable subtasks.

1. Flight agent: Search for flights from New York City to Los Angeles departing on May 13, 2025.
2. Hotel agent: Search for hotels in Los Angeles for the duration of the trip (check-in on May 13, check-out on May 16, 2025).
3. Excursion agent: Search for activities and excursions to do in Los Angeles during the stay (May 13-16, 2025).

Once these tasks are complete, I will summarize the findings.
---------- Flight_Agent ----------
[FunctionCall(id='call_SA2tTzf2bVkkxakMMRXTCnKQ', arguments='{"departure_date":"2025-05-13","return_date":"2025-05-16","source_airport_code":"JFK","destination_airport_code":"LAX"}', name='lookup_flight')]
---------- Flight_Agent ----------
[FunctionExecutionResult(content='H

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='I live near New York City. Book me a 3 day trip in Los Angeles. I am leaving on 5/13/2025.', type='TextMessage'), TextMessage(source='TravelPlanningAgent', models_usage=RequestUsage(prompt_tokens=153, completion_tokens=137), metadata={}, content='To plan your 3-day trip to Los Angeles starting on May 13, 2025, I will break down the tasks into smaller, manageable subtasks.\n\n1. Flight agent: Search for flights from New York City to Los Angeles departing on May 13, 2025.\n2. Hotel agent: Search for hotels in Los Angeles for the duration of the trip (check-in on May 13, check-out on May 16, 2025).\n3. Excursion agent: Search for activities and excursions to do in Los Angeles during the stay (May 13-16, 2025).\n\nOnce these tasks are complete, I will summarize the findings.', type='TextMessage'), ToolCallRequestEvent(source='Flight_Agent', models_usage=RequestUsage(prompt_tokens=333, completion_tokens